In [11]:
#default_exp query
%load_ext autoreload
%autoreload 2

In [1]:
#hide
import sys
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Queries

> Este módulo executa as queries sql / MongoDB necessárias para baixar os dados do STEL, RADCOM e MOSAICO

In [10]:
#export
from decimal import Decimal, getcontext
from typing import Union
from pathlib import Path
from urllib.request import urlopen, urlretrieve


import pandas as pd
import pyodbc
from fastcore.test import *
from rich.console import Console
from pyarrow import ArrowInvalid
import pandas_read_xml as pdx
from unidecode import unidecode
from fastcore.foundation import L
from fastcore.utils import listify


from anateldb.constants import *
from anateldb.format import optimize_objects, parse_bw, dict2cols
from anateldb.merge import clean_mosaico

getcontext().prec = 5

## Conexão com o banco de dados
A função a seguir é um `wrapper` simples que utiliza o `pyodbc` para se conectar ao banco de dados base da Anatel e retorna o objeto da conexão

In [3]:
#export
def connect_db():
    """Conecta ao Banco ANATELBDRO01 e retorna o 'cursor' (iterador) do Banco pronto para fazer iterações"""
    return pyodbc.connect(
        "Driver={ODBC Driver 17 for SQL Server};"
        "Server=ANATELBDRO01;"
        "Database=SITARWEB;"
        "Trusted_Connection=yes;"
        "MultipleActiveResultSets=True;",
        timeout=TIMEOUT,
    )

In [3]:
#slow
def test_connection():
    conn = connect_db()
    cursor = conn.cursor()
    for query in (RADCOM,):
        cursor.execute(query)
        test_eq(type(cursor.fetchone()), pyodbc.Row)

In [4]:
# test_connection()

In [4]:
#exporti
def _read_estações(path: Union[str, Path])->pd.DataFrame:
    """Read the zipped xml file `Estações.zip` from MOSAICO and returns a dataframe"""
    def extrair_ato(row):
        if not isinstance(row, str):
            row = listify(row)[::-1]
            for d in row:
                if not isinstance(d, dict):
                    continue
                if (d.get("@TipoDocumento") == "Ato") and (
                    d.get("@Razao") == "Autoriza o Uso de Radiofrequência"
                ):
                    return d["@NumDocumento"], d["@DataDOU"][:10]
            return "", ""
        return "", ""

    es = pdx.read_xml(path, ["estacao_rd"])
    dfs = []
    for i in range(es.shape[0]):
        df = pd.DataFrame(es["row"][i]).replace("", pd.NA)
        df = dict2cols(df)
        df.columns = [unidecode(c).lower().replace("@", "") for c in df.columns]
        dfs.append(df)
    df = pd.concat(dfs)
    df = df[df.state.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$")].reset_index(
        drop=True
    )
    docs = L(df.historico_documentos.apply(extrair_ato).tolist())
    df = df.loc[:, COL_ESTACOES]
    df["Num_Ato"] = docs.itemgot(0).map(str)
    df["Data_Ato"] = docs.itemgot(1).map(str)
    df.columns = NEW_ESTACOES
    df["Validade_RF"] = df.Validade_RF.astype("string").str.slice(0, 10)
    df["Data_Ato"] = df.Data_Ato.str.slice(0, 10)
    for c in df.columns:
        df.loc[df[c] == "", c] = pd.NA
    return df


def _read_plano_basico(path: Union[str, Path])->pd.DataFrame:
    """Read the zipped xml file `Plano_Básico.zip` from MOSAICO and returns a dataframe"""
    pb = pdx.read_xml(path, ["plano_basico"])
    dfs = []
    for i in range(pb.shape[0]):
        df = pd.DataFrame(pb["row"][i]).replace("", pd.NA)
        df = dict2cols(df)
        df.columns = [unidecode(c).lower().replace("@", "") for c in df.columns]
        dfs.append(df)
    df = pd.concat(dfs)
    df = df.loc[df.pais == "BRA", COL_PB].reset_index(drop=True)
    for c in df.columns:
        df.loc[df[c] == "", c] = pd.NA
    df.columns = NEW_PB
    df.sort_values(["Id", "Canal"], inplace=True)
    ENTIDADES.update(
        {r.Fistel: r.Entidade for r in df.itertuples() if str(r.Entidade) == "<NA>"}
    )
    df = df.groupby("Id", as_index=False).first()  # remove duplicated with NaNs
    df.dropna(subset=["Status"], inplace=True)
    df = df[df.Status.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$")].reset_index(
        drop=True
    )
    return df

In [6]:
# if (path := Path.cwd() / 'estações.zip').exists():
#     e = _read_estações(str(path))

In [5]:
from urllib.request import Request, urlopen
from urllib.error import URLError
req = Request(ESTACAO)
try:
    response = urlopen(req)
except URLError as e:
    if hasattr(e, 'reason'):
        print('We failed to reach a server.')
        print('Reason: ', e.reason)
    elif hasattr(e, 'code'):
        print('The server couldn\'t fulfill the request.')
        print('Error code: ', e.code)
else:
    Path.cwd().joinpath('estações.zip').write_bytes(response.read())

We failed to reach a server.
Reason:  [WinError 10060] Uma tentativa de conexão falhou porque o componente conectado não respondeu
corretamente após um período de tempo ou a conexão estabelecida falhou
porque o host conectado não respondeu


## Atualização das bases de dados
As bases de dados são atualizadas atráves das funções a seguir, o único argumento passado em todas elas é a pasta na qual os arquivos locais processados serão salvos, os nomes dos arquivos são padronizados e não podem ser editados para que as funções de leitura e processamento recebam somente a pasta na qual esses arquivos foram salvos.

In [14]:
#export
def formatar_tipos(df: pd.DataFrame, tipo: str = None)->pd.DataFrame:
    """Convert the dataframe from RADCOM to a dataframe with the correct types"""
    df['Frequência'] = df['Frequência'].astype('float')
    df['Latitude'] = df['Latitude'].astype('float32')
    df['Longitude'] = df['Longitude'].astype('float32')
    df['Entidade'] = df['Entidade'].astype('string')
    df['Fistel'] = df['Fistel'].astype('string')
    df['Município'] = df['Município'].astype('category')
    df['UF'] = df['UF'].astype('category')
    df['CNPJ'] = df['CNPJ'].astype('string')
    df['Número_da_Estação'] = df['Número_da_Estação'].astype('string')
    df["Num_Serviço"] = df['Num_Serviço'].astype("category")
    if tipo == 'radcom':
        a = df.Situação.isna()
        df.loc[a, 'Classe'] = df.loc[a, 'Fase']
        df.loc[~a, 'Classe'] = df.loc[~a, 'Fase'].astype('string') + '-' + df.loc[~a, 'Situação'].astype('string')
        df['Classe'] = df['Classe'].astype('category')
        df.drop(['Fase', 'Situação'], axis=1, inplace=True)
    return df

def update_radcom(pasta: Union[str, Path])->pd.DataFrame:
    """Atualiza a tabela local retornada pela query `RADCOM`"""
    console = Console()
    with console.status(
        "[cyan]Lendo o Banco de Dados de Radcom...", spinner="earth"
    ) as status:
        try:
            conn = connect_db()
            df = pd.read_sql_query(RADCOM, conn)
            # df = df_optimize(df, exclude=["Frequência"])
            df = formatar_tipos(df, 'radcom')
            try:
                df.to_feather(f"{pasta}/radcom.fth")
            except ArrowInvalid:
                Path(f"{pasta}/radcom.fth").unlink()
                df.to_excel(f"{pasta}/radcom.xlsx", engine="openpyxl", index=False)
            return df
        except pyodbc.OperationalError:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )
    return None    


def update_stel(pasta: Union[str, Path])->pd.DataFrame:
    """Atualiza a tabela local retornada pela query `STEL`"""
    console = Console()
    with console.status(
        "[red]Lendo o Banco de Dados do STEL. Processo Lento, aguarde...",
        spinner="moon",
    ) as status:
        try:
            conn = connect_db()
            df = pd.read_sql_query(STEL, conn)
            df["Validade_RF"] = df.Validade_RF.astype("string").str.slice(0, 10)
            df.loc[df.Unidade == "kHz", "Frequência"] = df.loc[
                df.Unidade == "kHz", "Frequência"
            ].apply(lambda x: Decimal(x) / Decimal(1000))
            df.loc[df.Unidade == "GHz", "Frequência"] = df.loc[
                df.Unidade == "GHz", "Frequência"
            ].apply(lambda x: Decimal(x) * Decimal(1000))
            df.drop('Unidade', axis=1, inplace=True)
            df = formatar_tipos(df, 'stel')
            try:
                df.to_feather(f"{pasta}/stel.fth")
            except ArrowInvalid:
                Path(f"{pasta}/stel.fth").unlink()
                df.to_excel(f"{pasta}/stel.xlsx", engine="openpyxl", index=False)
        except pyodbc.OperationalError:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )
    return df


def update_mosaico(pasta: Union[str, Path])->pd.DataFrame:
    """Atualiza a tabela local do Mosaico. É baixado e processado arquivos xml zipados da página pública do Spectrum E"""
    console = Console()
    with console.status(
        "[blue]Baixando as Estações do Mosaico...", spinner="shark"
    ) as status:
        stations, _ = urlretrieve(ESTACOES, f"{pasta}/estações.zip")
    with console.status(
        "[blue]Baixando o Plano Básico das Estações...", spinner="weather"
    ) as status:
        pb, _ = urlretrieve(PLANO_BASICO, f"{pasta}/canais.zip")
    console.print(":package: [blue]Consolidando as bases de dados...")
    estações = _read_estações(f"{pasta}/estações.zip")
    plano_basico = _read_plano_basico(f"{pasta}/canais.zip")
    df = estações.merge(plano_basico, on="Id", how="left")
    df.drop('Id', axis=1, inplace=True)
    # df["Número_da_Estação"] = df["Número_da_Estação"].fillna('-1')
    # df["Número_da_Estação"] = df["Número_da_Estação"].astype("string")
    df = clean_mosaico(pasta, df)
    try:
        df.reset_index(drop=True).to_feather(f"{pasta}/mosaico.fth")
    except ArrowInvalid:
        Path(f"{pasta}/mosaico.fth").unlink()
        with pd.ExcelWriter(f"{pasta}/mosaico.xlsx") as workbook:
            df.reset_index(drop=True).to_excel(
                workbook, sheet_name="Sheet1", engine="openpyxl", index=False
            )
#     Path(stations).unlink()
#     Path(pb).unlink()
    return df

def update_base(pasta: Union[str, Path])->pd.DataFrame:
    """Wrapper que atualiza opcionalmente lê e atualiza as três bases indicadas anteriormente, as combina e salva o arquivo consolidado na pasta `pasta`"""
    console = Console()
    stel = read_stel(pasta).loc[:, TELECOM]
    radcom = read_radcom(pasta).loc[:, SRD]
    mosaico = read_mosaico(pasta).loc[:, RADIODIFUSAO]
    radcom["Num_Serviço"] = "231"
    radcom["Status"] = "RADCOM"
    radcom["Classe_Emissão"] = pd.NA
    radcom["Largura_Emissão"] = BW_MAP['231']
#    filtro = radcom.Fase.notna() & radcom.Situação.notna()
#   radcom.loc[filtro, "Classe"] = radcom.loc[filtro, "Fase"].astype("string") + '-' + radcom.loc[filtro, "Situação"].astype("string")
    radcom["Entidade"] = radcom.Entidade.str.rstrip().str.lstrip()
    radcom["Num_Ato"] = pd.NA
    radcom["Data_Ato"] = pd.NA
    radcom["Validade_RF"] = pd.NA
    radcom["Fonte"] = "SRD"
    stel["Status"] = "L"
    stel["Num_Ato"] = pd.NA
    stel["Data_Ato"] = pd.NA
    stel["Entidade"] = stel.Entidade.str.rstrip().str.lstrip()
    stel["Fonte"] = "STEL"
    mosaico["Fonte"] = "MOS"
    mosaico["Classe_Emissão"] = pd.NA
    mosaico["Largura_Emissão"] = mosaico.Num_Serviço.map(BW_MAP)
#     mosaico = mosaico.loc[:, RADIODIFUSAO]
    # mosaico = df_optimize(mosaico, exclude=["Frequência"])
    rd = (
        pd.concat([mosaico, radcom, stel])
        .sort_values("Frequência")
        .reset_index(drop=True)
    )
    # rd["Num_Serviço"] = rd.Num_Serviço.astype("int")
    rd = rd.drop_duplicates(keep="first").reset_index(drop=True)
    rd['BW(kHz)'] = rd.Largura_Emissão.apply(parse_bw)
    rd = optimize_objects(rd, [])
    console.print(":trophy: [green]Base Consolidada. Salvando os arquivos...")
    try:
        rd.to_feather(f"{pasta}/base.fth")
    except ArrowInvalid:
        Path(f"{pasta}/base.fth").unlink()
        with pd.ExcelWriter(f"{pasta}/base.xlsx") as workbook:
            rd.to_excel(workbook, sheet_name="Sheet1", engine="openpyxl", index=False)
    return rd

In [15]:
pasta = Path.cwd().parent / 'dados' / 'tabular'

In [16]:
from anateldb.read import *
base = update_base(pasta)

🏆 Base Consolidada. Salvando os arquivos...

In [17]:
base

,Frequência,Entidade,Fistel,Número_da_Estação,Município,UF,Latitude,Longitude,CNPJ,Num_Serviço,Classe,Validade_RF,Status,Num_Ato,Data_Ato,Fonte,Classe_Emissão,Largura_Emissão,BW(kHz)
0,0.0280,FURNAS CENTRAIS ELETRICAS S A,01030052263,1557670,Nova Iguaçu,RJ,-22.662777,-43.476387,23274194000119,019,OP,2033-08-17,L,NaN,NaN,STEL,J9E,8K00,8.0
1,0.0285,COMPANHIA DE GERAÇÃO E TRANSMISSÃO DE ENERGIA ...,50420217282,1494686,Joinville,SC,-26.292500,-48.887222,02016507000169,019,OP,2025-08-31,L,NaN,NaN,STEL,R3E,2K50,2.5
2,0.0300,FURNAS CENTRAIS ELETRICAS S A,01030052263,859966,Araporã,MG,-18.410000,-49.099998,23274194000119,019,OP,2033-08-17,L,NaN,NaN,STEL,J3E,1K00,1.0
3,0.0300,FURNAS CENTRAIS ELETRICAS S A,01030052263,859761,Rio de Janeiro,RJ,-22.926666,-43.264999,23274194000119,019,OP,2033-08-17,L,NaN,NaN,STEL,J3E,500H,0.5
4,0.0300,FURNAS CENTRAIS ELETRICAS S A,01030052263,1557823,São Paulo,SP,-23.441668,-46.590832,23274194000119,019,OP,2033-08-17,L,NaN,NaN,STEL,J3E,1K00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884432,85469.0000,TIM S A,50417425295,1008754061,Goiânia,GO,-16.592699,-49.267799,02421421000111,019,FX,2032-08-31,L,NaN,NaN,STEL,Q7W,62M5,62500.0
884433,85469.0000,TIM S A,50417425295,1009125734,Pará de Minas,MG,-19.857389,-44.616112,02421421000111,019,FX,2032-08-31,L,NaN,NaN,STEL,Q7W,62M5,62500.0
884434,85469.0000,TIM S A,50417425295,1009786951,Ananindeua,PA,-1.358931,-48.385670,02421421000111,019,FX,2032-08-31,L,NaN,NaN,STEL,Q7W,750M,750000.0
884435,85469.0000,TIM S A,50417425295,1007183141,São Paulo,SP,-23.624083,-46.623943,02421421000111,019,FX,2032-08-31,L,NaN,NaN,STEL,Q7W,62M5,62500.0


In [12]:
base = pd.read_feather(pasta / 'base.fth')

In [13]:
base

,Frequência,Num_Serviço,Status,Classe,Entidade,Fistel,Número_da_Estação,Município,UF,Latitude,...,Num_Ato,Data_Ato,Classe_Emissão,Largura_Emissão,Fonte,Fase,Situação,CNPJ,Unidade,BW(kHz)
0,0.0280,19,L,OP,FURNAS CENTRAIS ELETRICAS S A,01030052263,1557670,Nova Iguaçu,RJ,-22.662777,...,-1,,J9E,8K00,STEL,NaN,NaN,<NA>,NaN,8.0
1,0.0285,19,L,OP,COMPANHIA DE GERAÇÃO E TRANSMISSÃO DE ENERGIA ...,50420217282,1494686,Joinville,SC,-26.292500,...,-1,,R3E,2K50,STEL,NaN,NaN,<NA>,NaN,2.5
2,0.0300,19,L,OP,FURNAS CENTRAIS ELETRICAS S A,01030052263,1558412,Mogi das Cruzes,SP,-23.709999,...,-1,,J3E,2K00,STEL,NaN,NaN,<NA>,NaN,2.0
3,0.0300,19,L,OP,FURNAS CENTRAIS ELETRICAS S A,01030052263,1557823,São Paulo,SP,-23.441668,...,-1,,J3E,1K00,STEL,NaN,NaN,<NA>,NaN,1.0
4,0.0300,19,L,OP,FURNAS CENTRAIS ELETRICAS S A,01030052263,859753,Campinas,SP,-22.774166,...,-1,,J3E,1K00,STEL,NaN,NaN,<NA>,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883494,85469.0000,19,L,FX,TIM S A,50417425295,1009125734,Pará de Minas,MG,-19.857389,...,-1,,Q7W,62M5,STEL,NaN,NaN,<NA>,NaN,62500.0
883495,85469.0000,19,L,FX,TIM S A,50417425295,1008775875,Fortaleza,CE,-3.734861,...,-1,,Q7W,62M5,STEL,NaN,NaN,<NA>,NaN,62500.0
883496,85469.0000,19,L,FX,TIM S A,50417425295,1009131726,Caruaru,PE,-8.266858,...,-1,,Q7W,62M5,STEL,NaN,NaN,<NA>,NaN,62500.0
883497,85469.0000,19,L,FX,TIM S A,50417425295,1005059940,Rio Branco,AC,-9.937445,...,-1,,Q7W,62M5,STEL,NaN,NaN,<NA>,NaN,62500.0


In [14]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883499 entries, 0 to 883498
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   Frequência         883499 non-null  float64 
 1   Num_Serviço        883499 non-null  int32   
 2   Status             883499 non-null  category
 3   Classe             880809 non-null  category
 4   Entidade           883481 non-null  category
 5   Fistel             883499 non-null  category
 6   Número_da_Estação  883499 non-null  int32   
 7   Município          883495 non-null  category
 8   UF                 883497 non-null  category
 9   Latitude           883499 non-null  float32 
 10  Longitude          883499 non-null  float32 
 11  Validade_RF        880727 non-null  category
 12  Num_Ato            869351 non-null  category
 13  Data_Ato           868393 non-null  category
 14  Classe_Emissão     883499 non-null  category
 15  Largura_Emissão    883499 non-null

In [15]:
# radcom = update_radcom(pasta)
radcom = pd.read_feather(pasta / 'radcom.fth')

In [16]:
radcom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4921 entries, 0 to 4920
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Frequência         4921 non-null   float64 
 1   Entidade           4921 non-null   string  
 2   Fistel             4921 non-null   string  
 3   Número_da_Estação  4921 non-null   string  
 4   Município          4921 non-null   category
 5   UF                 4921 non-null   category
 6   Latitude           4921 non-null   float32 
 7   Longitude          4921 non-null   float32 
 8   CNPJ               4921 non-null   string  
 9   Num_Serviço        4921 non-null   category
 10  Classe             4921 non-null   category
dtypes: category(4), float32(2), float64(1), string(4)
memory usage: 414.9 KB


In [17]:
radcom

,Frequência,Entidade,Fistel,Número_da_Estação,Município,UF,Latitude,Longitude,CNPJ,Num_Serviço,Classe
0,87.5,ASSOCIACAO COMUNITARIA VOZ DA LIBERDADE DE TUR...,50415095220,1008210959,Turilândia,MA,-2.228611,-45.306667,10910844000123,231,P
1,87.5,ASSOCIACAO COMUNITARIA DO MORAD.DE ALVORADA DE...,50409064718,699491851,Alvorada de Minas,MG,-18.734167,-43.364723,00635021000183,231,3
2,87.5,ASSOCIAÇÃO COMUNITARIA DA JUVENTUDE DE CONGONH...,50405625782,699359830,Congonhas do Norte,MG,-18.812778,-43.673611,04868495000126,231,3
3,87.5,ASSOCIACAO COMUNITARIA FOLHETA,50404381251,690859562,Dom Joaquim,MG,-18.950001,-43.266666,01809521000157,231,3
4,87.5,ASSOCIAÇÃO DE RÁDIO COMUNITÁRIA DE CASTANHEIRA...,50411566547,1008401606,Castanheira,MT,-11.137222,-58.613335,14970791000197,231,P
...,...,...,...,...,...,...,...,...,...,...,...
4916,107.9,ASSOCIAÇÃO DE MORADORES DO JARDIM CRISTINA OUR...,50434484237,692270272,São José dos Campos,SP,-23.553055,-45.870556,07249175000186,231,3
4917,107.9,ASSOCIAÇÃO DE MORADORES DO JARDIM CRISTINA OUR...,50406232911,692270272,São José dos Campos,SP,-23.553055,-45.870556,07249175000186,231,3
4918,107.9,ASSOCIAÇÃO DE MORADORES DO JARDIM CRISTINA OUR...,50413097013,692270272,São José dos Campos,SP,-23.553055,-45.870556,07249175000186,231,3
4919,107.9,ASSOCIACAO COMUNITARIA CULTURAL DE MUSICA E CI...,50406778205,693049723,São José dos Campos,SP,-23.191944,-45.875278,08894854000170,231,3


In [26]:
# stel = update_stel(pasta)

🌓  Lendo o Banco de Dados do STEL. Processo Lento, aguarde...

In [18]:
stel = pd.read_feather(pasta / 'stel.fth')

In [19]:
stel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852358 entries, 0 to 852357
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   Frequência         852358 non-null  float64 
 1   Classe_Emissão     852358 non-null  object  
 2   Largura_Emissão    852358 non-null  object  
 3   Classe             852358 non-null  object  
 4   Num_Serviço        852358 non-null  category
 5   Entidade           852358 non-null  string  
 6   Fistel             852358 non-null  string  
 7   Número_da_Estação  852358 non-null  string  
 8   Município          852358 non-null  category
 9   UF                 852358 non-null  category
 10  Latitude           852358 non-null  float32 
 11  Longitude          852358 non-null  float32 
 12  CNPJ               852358 non-null  string  
 13  Validade_RF        852358 non-null  string  
dtypes: category(3), float32(2), float64(1), object(3), string(5)
memory usage: 68.4+ MB


In [20]:
stel

,Frequência,Classe_Emissão,Largura_Emissão,Classe,Num_Serviço,Entidade,Fistel,Número_da_Estação,Município,UF,Latitude,Longitude,CNPJ,Validade_RF
0,10715.000,D7W,28M0,FX,053,TELEFONICA BRASIL S.A.,50403943221,690802021,Camaçari,BA,-12.773611,-38.193333,02558157000162,2023-06-29
1,10715.000,D7W,28M0,FX,053,TELEFONICA BRASIL S.A.,50403943221,688334695,Camaçari,BA,-12.660600,-38.335724,02558157000162,2023-06-29
2,10715.000,D7W,28M0,FX,053,TELEFONICA BRASIL S.A.,50403943221,683769804,Candeias,BA,-12.671111,-38.532501,02558157000162,2023-06-29
3,10715.000,D7W,29M6,FX,053,TELEFONICA BRASIL S.A.,50403943221,603851525,Itabuna,BA,-14.787778,-39.278610,02558157000162,2023-06-29
4,10715.000,D7W,28M0,FX,053,TELEFONICA BRASIL S.A.,50403943221,688412513,João Dourado,BA,-11.326944,-41.706390,02558157000162,2023-06-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852353,27.910,J3E,3K00,FX,019,UNIVERSIDADE FEDERAL RURAL DO RIO DE JANEIRO,50408283815,695826778,Seropédica,RJ,-22.783333,-43.690556,29427465000105,2021-08-25
852354,27.910,J3E,3K00,FX,019,UNIVERSIDADE FEDERAL RURAL DO RIO DE JANEIRO,50408283815,695831321,Seropédica,RJ,-22.787777,-43.683613,29427465000105,2021-08-25
852355,27.910,J3E,3K00,FX,019,UNIVERSIDADE FEDERAL RURAL DO RIO DE JANEIRO,50408283815,695831291,Seropédica,RJ,-22.791945,-43.681389,29427465000105,2021-08-25
852356,27.910,J3E,3K00,FX,019,UNIVERSIDADE FEDERAL RURAL DO RIO DE JANEIRO,50408283815,695831305,Seropédica,RJ,-22.789444,-43.664165,29427465000105,2021-08-25


In [29]:
mosaico = update_mosaico(pasta)

In [21]:
mosaico = pd.read_feather(pasta / 'mosaico.fth')

In [22]:
mosaico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29101 entries, 0 to 29100
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Num_Serviço               29101 non-null  object 
 1   Id                        29101 non-null  object 
 2   Número_da_Estação         22994 non-null  object 
 3   Latitude                  29101 non-null  float32
 4   Longitude                 29101 non-null  float32
 5   Validade_RF               26575 non-null  string 
 6   Num_Ato                   15006 non-null  object 
 7   Data_Ato                  14050 non-null  object 
 8   Município                 29098 non-null  string 
 9   Frequência                29101 non-null  float64
 10  Classe                    29084 non-null  object 
 11  Serviço                   29101 non-null  object 
 12  Entidade                  29083 non-null  object 
 13  UF                        29099 non-null  object 
 14  Status

In [23]:
mosaico.fillna('')

,Num_Serviço,Id,Número_da_Estação,Latitude,Longitude,Validade_RF,Num_Ato,Data_Ato,Município,Frequência,Classe,Serviço,Entidade,UF,Status,CNPJ,Fistel,Canal,Coordenadas_do_Município
0,248,57dbaad053c60,,-7.614167,-72.895836,,,,Mâncio Lima,539.0,C,TV,X-MEDIAGROUP S.A.,AC,TV-C1,03211814000163,50410887137,25,False
1,248,57dbaad0dc4e3,322647029,-12.101389,-44.993610,2023-12-31,6245,,Barreiras,79.0,A,TV,TELEVISAO OESTE BAIANO LTDA,BA,TV-C4,16395923000120,06030116240,5,False
2,248,57dbaad0eb54a,322623553,-14.779445,-39.262222,2023-12-31,3592,2021-05-28,Itabuna,69.0,A,TV,TELEVISAO SANTA CRUZ LTDA,BA,TV-C2,13476833000175,06020355110,4,False
3,248,57dbaad0ef8af,322623537,-14.781670,-39.261669,2023-12-31,7613,2018-10-24,Itabuna,177.0,B,TV,TV CABRALIA LTDA,BA,TV-C4,13494265000135,06020354903,7,False
4,248,57dbaad1077a6,637062230,-16.353056,-39.386112,2017-08-20,33606,2003-02-10,Porto Seguro,515.0,C,TV,FUNDACAO FUNDESUL,BA,TV-C7,04188244000109,50011828080,21,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29096,801,62828ff0ebc52,,-24.515301,-50.417599,,,,Tibagi,479.0,C,RTVD,TV NEWS - CANAL BRASILEIRO DE COMUNICACAO LTDA,PR,TV-C1,17962660000156,50442003609,15,False
29097,801,628bfacb2fc73,,-25.694401,-51.653599,,,,Pinhão,635.0,C,RTVD,TV NEWS - CANAL BRASILEIRO DE COMUNICACAO LTDA,PR,TV-C1,17962660000156,50442083963,41,False
29098,801,628bfbf7d2c2a,,-25.449200,-52.910198,,,,Quedas do Iguaçu,491.0,C,RTVD,TV NEWS - CANAL BRASILEIRO DE COMUNICACAO LTDA,PR,TV-C1,17962660000156,50442084005,17,False
29099,801,628f2587438e8,1000466032,-23.480860,-46.200249,2027-01-10,6863,2014-08-04,São Paulo,647.0,E,RTVD,REDE MULHER DE TELEVISAO LTDA,SP,TV-C4,02344518000178,50408927720,43,False


In [26]:
from anateldb.read import *
base = update_base(pasta)

🏆 Base Consolidada. Salvando os arquivos...

In [27]:
base

,Frequência,Entidade,Fistel,Número_da_Estação,Município,UF,Latitude,Longitude,CNPJ,Num_Serviço,Classe,Validade_RF,Status,Num_Ato,Data_Ato,Fonte,Classe_Emissão,Largura_Emissão,BW(kHz)
0,0.0280,FURNAS CENTRAIS ELETRICAS S A,01030052263,1557670,Nova Iguaçu,RJ,-22.662777,-43.476387,23274194000119,019,OP,2033-08-17,L,<NA>,<NA>,STEL,J9E,8K00,8.0
1,0.0285,COMPANHIA DE GERAÇÃO E TRANSMISSÃO DE ENERGIA ...,50420217282,1494686,Joinville,SC,-26.292500,-48.887222,02016507000169,019,OP,2025-08-31,L,<NA>,<NA>,STEL,R3E,2K50,2.5
2,0.0300,FURNAS CENTRAIS ELETRICAS S A,01030052263,859966,Araporã,MG,-18.410000,-49.099998,23274194000119,019,OP,2033-08-17,L,<NA>,<NA>,STEL,J3E,1K00,1.0
3,0.0300,FURNAS CENTRAIS ELETRICAS S A,01030052263,859761,Rio de Janeiro,RJ,-22.926666,-43.264999,23274194000119,019,OP,2033-08-17,L,<NA>,<NA>,STEL,J3E,500H,0.5
4,0.0300,FURNAS CENTRAIS ELETRICAS S A,01030052263,1557823,São Paulo,SP,-23.441668,-46.590832,23274194000119,019,OP,2033-08-17,L,<NA>,<NA>,STEL,J3E,1K00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884432,85469.0000,TIM S A,50417425295,1008754061,Goiânia,GO,-16.592699,-49.267799,02421421000111,019,FX,2032-08-31,L,<NA>,<NA>,STEL,Q7W,62M5,62500.0
884433,85469.0000,TIM S A,50417425295,1009125734,Pará de Minas,MG,-19.857389,-44.616112,02421421000111,019,FX,2032-08-31,L,<NA>,<NA>,STEL,Q7W,62M5,62500.0
884434,85469.0000,TIM S A,50417425295,1009786951,Ananindeua,PA,-1.358931,-48.385670,02421421000111,019,FX,2032-08-31,L,<NA>,<NA>,STEL,Q7W,750M,750000.0
884435,85469.0000,TIM S A,50417425295,1007183141,São Paulo,SP,-23.624083,-46.623943,02421421000111,019,FX,2032-08-31,L,<NA>,<NA>,STEL,Q7W,62M5,62500.0


In [28]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884437 entries, 0 to 884436
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Frequência         884437 non-null  float64
 1   Entidade           884419 non-null  object 
 2   Fistel             884437 non-null  object 
 3   Número_da_Estação  878330 non-null  object 
 4   Município          884434 non-null  object 
 5   UF                 884435 non-null  object 
 6   Latitude           884437 non-null  float32
 7   Longitude          884437 non-null  float32
 8   CNPJ               884437 non-null  object 
 9   Num_Serviço        884437 non-null  object 
 10  Classe             884420 non-null  object 
 11  Validade_RF        876992 non-null  object 
 12  Status             884437 non-null  object 
 13  Num_Ato            15005 non-null   object 
 14  Data_Ato           14049 non-null   object 
 15  Fonte              884437 non-null  object 
 16  Cl

In [31]:
from anateldb.format import optimize_objects
base = optimize_objects(base, datetime_features=[])
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884437 entries, 0 to 884436
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   Frequência         884437 non-null  float64 
 1   Entidade           884419 non-null  category
 2   Fistel             884437 non-null  category
 3   Número_da_Estação  878330 non-null  category
 4   Município          884434 non-null  category
 5   UF                 884435 non-null  category
 6   Latitude           884437 non-null  float32 
 7   Longitude          884437 non-null  float32 
 8   CNPJ               884437 non-null  category
 9   Num_Serviço        884437 non-null  category
 10  Classe             884420 non-null  category
 11  Validade_RF        876992 non-null  category
 12  Status             884437 non-null  category
 13  Num_Ato            15005 non-null   category
 14  Data_Ato           14049 non-null   category
 15  Fonte              884437 non-null

In [34]:
base.to_feather(pasta / 'base.fth')

,Frequência,Entidade,Fistel,Número_da_Estação,Município,UF,Latitude,Longitude,CNPJ,Num_Serviço,Classe,Validade_RF,Status,Num_Ato,Data_Ato,Fonte,Classe_Emissão,Largura_Emissão,BW(kHz)
0,0.0280,FURNAS CENTRAIS ELETRICAS S A,01030052263,1557670,Nova Iguaçu,RJ,-22.662777,-43.476387,23274194000119,019,OP,2033-08-17,L,NaN,NaN,STEL,J9E,8K00,8.0
1,0.0285,COMPANHIA DE GERAÇÃO E TRANSMISSÃO DE ENERGIA ...,50420217282,1494686,Joinville,SC,-26.292500,-48.887222,02016507000169,019,OP,2025-08-31,L,NaN,NaN,STEL,R3E,2K50,2.5
2,0.0300,FURNAS CENTRAIS ELETRICAS S A,01030052263,859966,Araporã,MG,-18.410000,-49.099998,23274194000119,019,OP,2033-08-17,L,NaN,NaN,STEL,J3E,1K00,1.0
3,0.0300,FURNAS CENTRAIS ELETRICAS S A,01030052263,859761,Rio de Janeiro,RJ,-22.926666,-43.264999,23274194000119,019,OP,2033-08-17,L,NaN,NaN,STEL,J3E,500H,0.5
4,0.0300,FURNAS CENTRAIS ELETRICAS S A,01030052263,1557823,São Paulo,SP,-23.441668,-46.590832,23274194000119,019,OP,2033-08-17,L,NaN,NaN,STEL,J3E,1K00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884432,85469.0000,TIM S A,50417425295,1008754061,Goiânia,GO,-16.592699,-49.267799,02421421000111,019,FX,2032-08-31,L,NaN,NaN,STEL,Q7W,62M5,62500.0
884433,85469.0000,TIM S A,50417425295,1009125734,Pará de Minas,MG,-19.857389,-44.616112,02421421000111,019,FX,2032-08-31,L,NaN,NaN,STEL,Q7W,62M5,62500.0
884434,85469.0000,TIM S A,50417425295,1009786951,Ananindeua,PA,-1.358931,-48.385670,02421421000111,019,FX,2032-08-31,L,NaN,NaN,STEL,Q7W,750M,750000.0
884435,85469.0000,TIM S A,50417425295,1007183141,São Paulo,SP,-23.624083,-46.623943,02421421000111,019,FX,2032-08-31,L,NaN,NaN,STEL,Q7W,62M5,62500.0


In [10]:
from nbdev.export import notebook2script; notebook2script()

Converted Apresentação REFIS 2022 - anateldb.ipynb.
Converted constants.ipynb.
Converted filter.ipynb.
Converted format.ipynb.
Converted index.ipynb.
Converted merge.ipynb.
Converted query.ipynb.
Converted read.ipynb.
